# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'
cache_glm = '/root/autodl-tmp/cache/THUDM/chatglm3-6b/'

In [2]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [3]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
base_model = AutoModel.from_pretrained(model_name_or_path,
                                       cache_dir=cache_glm,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, cache_dir=cache_glm, trust_remote_code=True)

## 使用微调前 ChatGLM3

In [6]:
input_text = "解释下乾卦是什么？"

In [7]:
response, history = base_model.chat(tokenizer, query=input_text)

In [8]:
print(response)

乾卦是《易经》中的第一卦，也是八卦之一。乾卦是由两个阴爻夹一个阳爻构成，象征着天、云、雷等自然现象。乾卦的卦辞是“元、亨、利、贞”，意味着“开始、发展、顺利、正道”。

乾卦的阳爻代表阳刚之气，阴爻代表阴柔之气。在《易经》中，阳爻和阴爻具有相对的性质，相互依存、相互制约。乾卦中的三个爻分别代表了不同的层次，由下至上，形成了一个等级秩序。

乾卦具有以下象征意义：

1. 天：乾卦象征着天、云、雷等自然现象，以及刚强、积极、行动力等品质。
2. 领导：乾卦代表领导者的形象，象征着权威、刚毅、坚定等品质，具有指导、引领、统御的能力。
3. 刚强：乾卦具有刚强、积极、行动力等品质，象征着男子、力量、进展等。
4. 变化：乾卦的变化象征着事物的发展变化，表明事物是不断变化、发展的。

在《易经》的六十四卦中，乾卦具有重要的地位，是八卦的基础和核心。它不仅反映了自然界的现象和变化，也反映了人类社会的伦理、道德、政治、经济等方面的变化。


In [9]:
response, history = base_model.chat(tokenizer, query="地水师卦是什么？", history=history)
print(response)

地水师卦是《易经》中第63卦。它的卦辞是“元、亨、利、贞”，意味着“开始、发展、顺利、正道”。师卦是由两个阴爻夹一个阳爻构成，象征着地、水等自然现象。

地水师卦具有以下象征意义：

1. 地：地水师卦象征着地、土壤、大地等自然现象，以及柔顺、承载、承载等品质。
2. 众：地水师卦代表众、众数、众多等含义，象征着人们聚集在一起，共同完成某个目标。
3. 顺从：地水师卦象征着顺从、听从、遵循等品质，表示人们应该尊重自然、尊重规律，顺应事物的发展变化。
4. 承载：地水师卦具有承载、包容、支持等品质，象征着人们应该相互扶持，共同承担责任。

在《易经》的六十四卦中，地水师卦的位置较为特殊，位于互传卦（互为八卦）的序列中。它与乾卦、坤卦、震卦、巽卦、坎卦、离卦、艮卦、兑卦相互对应，共同构成了八卦。这些八卦在《易经》中具有重要的象征意义，不仅反映了自然界的现象和变化，也反映了人类社会的伦理、道德、政治、经济等方面的变化。


## 微调前后效果对比

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [10]:
from peft import PeftModel, PeftConfig

epochs = 50
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

### 使用微调后的 ChatGLM3-6B

In [11]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B微调后：\n{ft_response}")
    return base_response, ft_response

In [12]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是《易经》中的第一卦，也是六十四卦中的第一卦，象征着天、云、龙等阳性事物。乾卦是由两个阳爻夹一个阴爻构成，阴爻位于阳爻之上，表示一种 hierarchical structure。乾卦的含义包括强大、刚健、威严、权威、道义等，它代表了一种刚健不挠的精神，是人们追求进步和发展的精神象征。在《易经》中，乾卦的卦辞是“元、亨、利、贞”，表示追求进步和发展的愿望，能够得到成功和幸福，是一种理想的状态。在卦象上，乾卦为九阳一阴，象征天、云、龙等阳性事物，意味着刚强、权威和威严。


ChatGLM3-6B微调后：
[gMASK]sop 解释下乾卦是什么？乾卦是《易经》中的第一卦，也是六十四卦中的首卦。乾卦是由两个阴爻夹一个阳爻构成，象征着天。乾卦的阳爻在下面，阴爻在上面，表示一种阳上升、阴下降的宇宙观。乾卦象征着刚强、独立、强大、积极向上，也象征着事物的发展和变化。在《易经》中，乾卦代表天，也代表君主，它反映了君主的意愿，以及君主的行为。乾卦的卦辞是：“元亨，利贞。象曰：天行健，君子以自强不息。”乾卦的卦辞象征天，它反映了宇宙的运行和事物的生长，也象征着君主的统治和国家的治理。


In [13]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
师卦是《易经》中的第六卦，卦象为：乾（天），坤（地）。”师“字的意思为军队的统帅，故《易经》中师卦象征军队，乾为阳，为兵，坤为顺，表示军队顺从指挥，协调一致。师卦寓意着“师”，即军队，也象征军队的“顺”，表示顺从和协调。在《易经》中，师卦是阳爻居下位，阴爻居上位，阴顺从阳，表示顺应自然，以柔顺刚，协调和谐，积极进攻，积极入世。


ChatGLM3-6B微调后：
[gMASK]sop 地水师卦是什么？ 师卦是《易经》中的卦象之一，卦象是由下卦（坎）上卦（坤）所组成。师卦象征军队，表示军事实力和军事实地的统一，也象征军队的众和师长的尊。在《易经》中，师卦是“师”字卦，表示众，尊师重道，强调整体协调，共同完成任务。


In [14]:
base_response, ft_response = compare_chatglm_results(query="天水讼卦")

问题：天水讼卦

原始输出：
讼卦（卦名：讼）是《易经》卦象之一，卦象是：雷综述，天雷 performances，象征 Performs a dramatic performance， stage play， show， performance， etc. 居于坤位，复卦为乾，得乾为健的时运，格卦为震，得震则观谋，行动以行，只管行动。

讼卦象意解析：象征诉讼，也象征 confusion 和 矛盾，background: #f2f2f2; text-align: center; font-size: 18px;">卦象：雷 坤：君子时中，利见大人，象鼓吹 music for a dried-up field，天养万物，物悦于德，乐行天下，动，行德。坤为子，为布，为养，为政，宜散布德行，顺服众，众则变，变则思，思则明，明则警，警则行。

坎上坤下：讼。

讼：有争诉，有争议，象征吉祥。

《易经》解释：讼卦象征辩论，象征诉讼，也象征 confusion 和 矛盾。

卦象：下坎上坤

《象辞》解释：下坎象征水，上坤象征地，水行于地之上，象征顺从。

卦义：诉讼，争议， confusion，矛盾，矛盾可致吉祥，需化解，需合。

卦象：下坎上坤

卦宫：坤

卦险：格：致，致德

卦象：雷：君子

断卦：君子争诉，利见大人，德行存于心，德则思，思则明，明则警，警则行。


ChatGLM3-6B微调后：
[gMASK]sop 天水讼卦 讼卦原文：讼。有孚，窒惕，中吉，终凶。利见大人，不利涉大川。象曰：天与水违行，讼。君子以做事谋始。白话文解释：讼卦象征虽有利可图但需警惕。事情初吉后凶，利于见贵人，不宜涉水。《象辞》说：上卦为乾（天），下卦为坎（水），天水相隔，事理不合，君子需慎重谋事。《断易天机》解：讼卦乾上坎下，刚遇险，必有争论，多不吉。北宋易学家邵雍解：天高水深，远离不亲，慎谋退守则无凶。得此卦者，身心不安，多争诉，宜修身养性。台湾国学大儒傅佩荣解：时运受阻，财运初谨慎终获利，家宅君子求淑女，身体预防胜于治疗。传统解卦：异卦（下坎上乾），刚健遇险，彼此反对，生争讼，需慎重戒惧。


#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [15]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [16]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B微调后：\n{ft_response}")
    return base_response, ft_response

In [17]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，包含着传达天意的意思。它的卦象由六个阳爻组成，排列成阳阳相生的形势，象征着天地的运行和生育。在卜问中，阳爻表示刚强，阴爻表示柔顺，因此乾卦代表着刚强果敢、积极进取的态度。在运势方面，求得者会得到 timing，有利于阳谋大计。

乾卦的卦辞这样解释：

天行：代表天地的运行

健志：象征刚强果敢

口感：表示积极进取

事情：有利于阳谋大计

运势：运势顺利

事业：权势地位

事业：先声夺人 

运势：刚强果敢

事业：权势地位

事业：先声夺人 

事业：权势地位

事业：先声夺人 

运势：运势顺利

事业：得志如灰

事业：得志如灰

运势：运势顺利

事业：得志如灰

事业：得志如灰

运势：运势顺利

事业：得志如灰

运势：运势顺利

事业：得志如灰

乾卦的运势特点：

初爻：径直

二爻：强健

三爻：奔走

四爻：得志

事业：权势地位

事业：先声夺人

事业：得志如灰

事业：得志如灰

运势：运势顺利

事业：得志如灰

事业：得志如灰

运势：运势顺利

事业：得志如灰

运势：运势顺利

事业：得志如灰

运势：运势顺利

事业：得志如灰

运势：运势顺利

事业：得志如灰

乾卦的事业特点：

刚强果敢，积极进取

权势地位，权宜利用

先声夺人，先发制人

先声夺人，先发制人

得志如灰，内心满足

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

得志如灰，内心满足

先声夺人，先发制人

得志如灰，内心满足

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，先发制人

先声夺人，

In [18]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
地水师卦是一个由坎卦（水）上承坤卦（地）组成的卦象，代表地下的水流与地下的土壤相交，象征恩惠和力量在下面得以成就。在《易经》中，地水师卦被认为是一种吉祥的卦象，具有决断和行动的能力。

在卜筮时，得到此卦意味着 index+=I，其中 I 表示当前指数。初始指数为 0，代表着一种积极、主动、充满决断力的态度。然而，如果 index 出现 negative 值，则可能表示遇到了困难或挫折。

地水师卦的决策过程需要综合考虑 both 上卦（坎）和下卦（坤），象征需要冷静、沉著、谨慎行事。在决策过程中，需要注重沟通和协作，以及充分准备和计划。

地水师卦的解释需要结合《象辞》和《决卦》来分析，解释时需要考虑到 both 卦象的含义和象征意义。在解释过程中，需要注重当前形势和环境，以及决策者的态度和行动。

总结起来，地水师卦是一种充满决断力和力量在下的卦象，具有积极、主动的态度。然而，在决策过程中，需要充分考虑环境和形势，以及决策者的态度和行动。


ChatGLM3-6B微调后：
[gMASK]sop 地水师卦是什么？ 地水师卦是一个由坎卦（水）上承坤卦（地）组成的卦象，代表一种以地载水的情况。在卜筮时，吉凶由卜者所持之念决定。

《易经》中解释说：“地水师卦：时运运，当大吉利。”

这是一个吉祥的卦象，预示着卜者将会遇到一些有利于水的地，可以得到他人的帮助。

哲学家们解释说：地水师卦象征着地载水的情况，提示着在人生道路上要遇到困难和阻碍，需要以柔克刚，以静制动，以柔克刚，以静制动，以柔克刚的策略来应对，从而实现愿望。

运势方面：

地水师卦：时运运，大吉利。

指数：90/100

指数说明：指数在0-100之间，表示卜者运势的状况。90/100的指数表示卜者运势非常顺利，将会得到他人的帮助，事业、财运、感情等方面都有所收获。

事业：

地水师卦：时运运，大吉利。

指数：90/100

指数说明：指数在0-100之间，表示卜者运势的状况。90/100的指数表示卜者运势非常顺利，将会得到他人的帮助，事业、财运、感情等方面都有所收获。

财运：

地水师卦：时运运，大吉利。

指数：90/100

指数说明：指数在0-100之间，表示卜者运势的状况。90/100的指数表示卜者运势非常顺利，将会得到他人的帮助，事业、财运、感情

In [19]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")

问题：周易中的讼卦是什么

原始输出：
讼卦是周易卦象中的一卦，代表诉讼和争端的状况。卦象为乾（天）卦象上（天）卦下，具有天地的象数，具有刚健之志。讼卦的特性是天时与地时的相互作用，预示着 conflict（诉讼）可能发生。在讼卦中，天时（阳）为不利，地时（阴）为不利，并且双方都愿出口，但天时不利，地时也不利，说明双方都带有刚健之志，但情况并不乐观。

讼卦的核心哲学是“天时与地时”，强调不利，代表着不利的情况。卦象中的天时为不利，地时也为不利，意味着争端难以达成，两者相互抑制，预示着 conflict（争端）难以解决。

讼卦的占卜结果显示，在争端中，双方都有强烈的愿望和决心，但情况并不利，容易引起争端，难以达成共同目标。因此，讼卦提示人们，在处理争端和冲突时应谨慎，尽量以柔克刚，避免过于刚健，从而求得利导。"

#讼卦 #周易 #哲学


ChatGLM3-6B微调后：
[gMASK]sop 周易中的讼卦是什么卦象

 讼卦是周易中的一卦，代表诉讼和争端的卦象。卦象为：上卦为乾（天），下卦为坎（水），两者相背而行，代表天与水违行的状况，象征着事理乖舛和争端的危险。讼卦中有利可图，但必须警惕戒惧，事情中间吉利，但最终会有凶险。在卜卦时，利于会见贵族王公，但不利于涉水渡河。

讼卦的核心哲学是：开始可能顺利，有所收获，但随后会遇到困难和挫折。因此，务必慎之又慎，不得固执已见，避免介入诉讼纠纷的争执之中。退让而不固执，求得化解，安于正理，可免除意外之灾。陷入争端，即使获胜，最后还得失去，得不偿失。

讼卦的经商指引是：和气生财，吃亏是福，切勿追求不义之财。在商业谈判中要坚持公正、公平、互利的原则，尽量避免发生冲突。

对于决策，讼卦提醒我们，争强好胜，不安于现状，为改变命运和超越他人而奋斗。但缺乏持之以恒的毅力，容易得罪他人，带来诉讼之灾。因此，接受教训，引以为戒，可功成名就。

讼卦所蕴含的智慧是：在面对争端和异见时，要善于退让和求和，坚守正道，谨慎处事，以避免不必要的冲突和损失。


#### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed))

In [22]:
from peft import PeftModel, PeftConfig

epochs = 3
timestamp = "20240120_000357"
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [23]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：\n{ft_response}")
    return base_response, ft_response

In [24]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦是八卦之一，由两个阳爻夹一个阴爻组成，象征着天。它代表着刚健、健行、刚健不屈的意境。乾卦所代表的是干燥、坚硬、 tail(狼尾) 的高山，以及天顶。 在股市中，乾卦代表着投资者的精神状态，即坚定信念，不受外界干扰，努力前进。乾卦也代表着冷静、沉着、坚定和果断，以及在股市中表现出冷静、沉着、坚定和果断的品质。', 'content': '\n乾卦的哲学内含是：刚健不屈，刚健健行，干燥耐寒，坚韧不拔。在人生中，坚贞不屈的品格是乾卦所代表的。在事业中，乾卦代表着刚健、坚贞、公正、诚信，以及力量Display\n\n乾卦的运势：在事业方面，乾卦代表着稳定、严肃、公正、坚定和刚健。在事业中，象征着事业稳定、严肃、公正、坚定和刚健的状况。在经商中，乾卦代表着刚健、坚贞、公正、诚信，以及力量 Display。在婚姻方面，乾卦代表着刚健、坚贞、公正、诚信，以及力量 Display。在决策中，乾卦代表着刚健、坚贞、公正、诚信，以及力量 Display。\n\n總之，乾卦代表着刚健、坚贞、公正、诚信，以及力量 Display，在事业、经商、婚姻和决策中都具有重要的意义。'}


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [25]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
地水师卦，又称地水卦，是一个由地爻和水面爻组成的卦象。它由下卦艮卦（山卦）和上卦坎卦（水卦）组成。在这个卦象中，地象征稳定和坚守，而水象征变化和成长。因此，地水师卦象征稳定和坚守，同时注重变通和成长。

地水师卦的卦象充满了象征意义，例如山象征坚定和刚毅，水象征柔顺和柔弱。它也象征着困难与困境，以及需要耐心和智慧去克服。然而，地水师卦并非完全是一个 negative的卦象，它也代表着一种稳定的力量和灵活的思考。

地水师卦的卦象象征着，在面临困难与困境时，需要保持冷静和坚定，坚守信念，同时注重变通和成长。在解决问题和决策时，需要注重 steady progress（稳定进步），并灵活应对。地水师卦的核心哲学是：坚定信念，稳步前行，注重变通，迎接挑战。


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 地水师卦是什么？ 地水师卦是一个由下卦、上卦和卦象组成的卦象，它是由坎卦（水）和坤卦（地）相叠而成的。在这个卦象中，地象征大地的稳定和宁静，坎象征 flexibility and adaptability. 君子观此卦象，可以得到灵活应变的智慧，以便解决各种问题。

对于事业， this卦象表明，事业上有稳定和发展的趋势。然而，也应注意适应环境的变化，并要警惕潜在的危险。对于经商，卦象显示，经商过程中应提示灵活变通，並需警惕危险。对于求名，卦象强调，应得贵人的帮助而能取得成功。对于决策，卦象强调，应谨慎行事，避免盲目行动。

地水师卦是一个提示，在事业、经商、求名和决策方面都需要灵活变通，但也需警惕潜在危险。


In [26]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 周易中的讼卦是什么样子 讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。
